In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
def f_ativacao(x):
    """ Funcao de ativacao sigmoidal """
    return (1 / (1 + np.exp(-x)))

def df_ativacao(x):
    """ Derivada da funcao de ativacao """
    return (x * (1-x))

In [3]:
def arquitetura(tam_entrada=2, tam_escondida=2, tam_saida=1,
                f_ativacao=f_ativacao, df_ativacao=df_ativacao):
    """ Cria a arquitetura da MLP """
    
    arq = {'tam_entrada' : tam_entrada, 
           'tam_escondida' : tam_escondida,
           'tam_saida' : tam_escondida,
           'f_ativacao' : f_ativacao,
           'df_ativacao' : df_ativacao,
           }
    
    tam_pesos_saida = (tam_escondida+1) * tam_saida
    tam_pesos_escondida = (tam_entrada+1) * tam_escondida
    
    # Inicializacao com pesos aleatórios de -0.5 a 0.5
    arq['pesos_saida'] = np.atleast_2d(np.random.uniform(low=-0.5,
                                                         high=0.5,
                                                         size=tam_pesos_saida
                                                         ).reshape(tam_saida,tam_escondida+1))
    
    arq['pesos_escondida'] = np.atleast_2d(np.random.uniform(low=-0.5,
                                                             high=0.5,
                                                             size=tam_pesos_escondida
                                                             ).reshape(tam_escondida,tam_entrada+1))
    
    return arq

In [4]:
def forwardpropagation(arq, exemplo):
    """ Computa a saida h(x) da rede para o exemplo x """
    
    X_escondida = np.atleast_2d(arq['pesos_escondida'] @ np.append(exemplo, 1))
    Y_escondida = arq['f_ativacao'](X_escondida)
    
    X_saida = np.atleast_2d(arq['pesos_saida'] @ np.append(Y_escondida, 1))
    Y_saida = arq['f_ativacao'](X_saida)
   
    resultados = {'X_escondida': X_escondida,
                  'Y_escondida': Y_escondida,
                  'X_saida': X_saida,
                  'Y_saida': Y_saida,}
    
    return resultados

In [5]:
def backpropagation(arq, dados, eta=0.1, limiar=1e-3, maxit=np.inf):
    """ Ajusta os pesos sinapticos de acordo com o erro produzido por um exemplo """
    erroquadratico = 2*limiar
    epocas = 0
        
    while erroquadratico > limiar and epocas < maxit:
        erroquadratico = 0
        
        #Uma época completa:
        for i in range(len(dados.index)):  
            
            #extração de um exemplo i de entrada dentre o dataset inteiro:
            exemplo = np.asarray(dados.iloc[i, 0:arq['tam_entrada']])
            #extração do valor que deverá ser obtido ao final da época
            desejado = np.asarray(dados.iloc[i, arq['tam_entrada']: len(dados.columns)])
            
            resultados = forwardpropagation(arq, exemplo)
            obtido = resultados['Y_saida']
            
            erro = desejado - obtido
            
            erroquadratico = erroquadratico + np.sum(erro**2)
            
            grad_local_saida = np.atleast_2d(erro * arq['df_ativacao'](obtido))
            
            #recorte dos pesos de saída EXCLUIDO O BIAS
            pesos_saida = arq['pesos_saida'][:,0:arq['tam_escondida']]
            
            grad_local_escondida = np.asarray(arq['df_ativacao'](resultados['Y_escondida'])) * (
                                   np.asarray(grad_local_saida) @ pesos_saida)
            
            #Treino dos pesos sinápticos:
            arq['pesos_saida'] = arq['pesos_saida'] + eta * (
                                 np.transpose(grad_local_saida) @ (
                                 np.atleast_2d(np.append(resultados['Y_escondida'],1))))
            
            arq['pesos_escondida'] = arq['pesos_escondida'] + eta * (
                                     np.transpose(grad_local_escondida) @ (
                                     np.atleast_2d(np.append(exemplo,1))))
        
        #Obtenção do Erro quadrático médio
        erroquadratico = erroquadratico / len(dados.index)
        
        if (epocas % 10 == 0): # (printa o erro a cada 10 epocas)
            print("Erro Quadrático Médio:", erroquadratico, "Epoca:", epocas)
            
        epocas += 1
    
    print("\nErro Quadrático Médio Final:", erroquadratico, "\nUltima epoca:", epocas)
    return arq

In [6]:
def change_value(dados):
    """Essa função trocará os valores 'target' do dataset iris
       para n valores binários sendo n o numero de classes originais.
       No caso, setosa = [1, 0, 0]; versicolor = [0, 1, 0]; virginica = [0, 0, 1]
    """
    n1 = list()
    n2 = list()
    n3 = list()
    for i in range(len(dados.index)):
        if dados['target'][i] == 0.0:
            n1.append(1)
            n2.append(0)
            n3.append(0)

        elif dados['target'][i] == 1.0:
            n1.append(0)
            n2.append(1)
            n3.append(0)
        elif dados['target'][i] == 2.0:
            n1.append(0)
            n2.append(0)
            n3.append(1)
        
    values = dict()
    values['n1'] = n1
    values['n2'] = n2
    values['n3'] = n3

    return (values)

In [7]:
#Carregamento do dataset iris
iris = datasets.load_iris()
dados = pd.DataFrame(data=np.c_[iris['data'], iris['target']], 
                     columns= iris['feature_names'] + ['target'])
dados.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [8]:
# Mudando o valor 'target' do dataset:  
new_values = change_value(dados)
n1 = np.array(new_values['n1'])
n2 = np.array(new_values['n2'])
n3 = np.array(new_values['n3'])

dados = pd.DataFrame(data=np.c_[iris['data'], n1,n2,n3],
                     columns=iris['feature_names'] + ['n1','n2','n3'])
dados.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),n1,n2,n3
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0


In [9]:
# Separando os dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(dados.iloc[:,:4], dados.iloc[:,4:], 
                                                    test_size=0.33, random_state=0)

In [10]:
# Criação de uma rede neural com 4 neuronios de entrada, 3 escondidos e 3 de saída 
arq = arquitetura(4,3,3)
arq

{'tam_entrada': 4,
 'tam_escondida': 3,
 'tam_saida': 3,
 'f_ativacao': <function __main__.f_ativacao(x)>,
 'df_ativacao': <function __main__.df_ativacao(x)>,
 'pesos_saida': array([[-0.2153819 , -0.16944775, -0.11495614, -0.28465744],
        [ 0.09907864,  0.39028259, -0.38339913,  0.31155623],
        [-0.08090527, -0.46339914,  0.09062457,  0.06567107]]),
 'pesos_escondida': array([[-0.13788932, -0.18905951,  0.27540271,  0.00110897,  0.17698265],
        [-0.08592479, -0.11388623, -0.28409418, -0.04190437,  0.22608866],
        [-0.07489272, -0.09072576,  0.25773385, -0.38559087, -0.04714499]])}

In [11]:
# Treinamento dos pesos sinápticos
model = backpropagation(arq, pd.DataFrame(data=np.c_[X_train,y_train]),
                        eta=0.1,limiar=1e-2,maxit=100)

Erro Quadrático Médio: 0.6952229757168794 Epoca: 0
Erro Quadrático Médio: 0.3726439588671813 Epoca: 10
Erro Quadrático Médio: 0.29275918100143666 Epoca: 20
Erro Quadrático Médio: 0.22062350478173628 Epoca: 30
Erro Quadrático Médio: 0.1609172472950574 Epoca: 40
Erro Quadrático Médio: 0.12192504367419352 Epoca: 50
Erro Quadrático Médio: 0.09888278670726072 Epoca: 60
Erro Quadrático Médio: 0.08521339370573218 Epoca: 70
Erro Quadrático Médio: 0.07627462013867091 Epoca: 80
Erro Quadrático Médio: 0.06980966674475911 Epoca: 90

Erro Quadrático Médio Final: 0.06538591509780284 
Ultima epoca: 100


In [12]:
# Pesos finais da camada escondida
model['pesos_escondida']

array([[-5.98150803e-01, -1.82069078e+00,  2.99178662e+00,
         1.27135523e+00, -1.48374099e-01],
       [ 3.46776889e+00,  2.23565561e+00, -4.78259220e+00,
        -3.49582098e+00,  2.50907913e+00],
       [ 1.94799837e-01,  7.10115482e-01, -4.42490146e-03,
        -2.44125946e-01, -3.52962296e-02]])

In [13]:
# Pesos finais da camada de saida
model['pesos_saida'] 

array([[-5.15667988,  2.58422752, -0.07923324,  0.1662553 ],
       [ 4.1985014 ,  4.09324579, -3.399099  , -3.00259481],
       [ 1.97469162, -5.58213742,  0.57532649,  0.09379902]])

Agora, com os pesos sinápticos treinados, é possível predizer o output correto de uma certa entrada:

In [14]:
# Exemplo: Qual a classe do primeiro exemplo dos dados de teste?
np.around(forwardpropagation(model, X_test.iloc[0])['Y_saida'])

array([[0., 0., 1.]])

In [15]:
# Obtendo o resultado para todos os casos de teste
results = pd.DataFrame()
for i in range(len(X_test)):
    test = forwardpropagation(model, X_test.iloc[i])
    test = pd.DataFrame(np.around(test['Y_saida']))
    results = pd.concat([results,test])
    
results.head()

,0,1,2
0,0.0,0.0,1.0
0,0.0,1.0,0.0
0,1.0,0.0,0.0
0,0.0,0.0,1.0
0,1.0,0.0,0.0


In [16]:
# Desempenho da rede:
accuracy_score(y_test, results)

0.92